In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter

# Parameters
L = 20  # Lattice size (LxL)
J = 1.0  # Interaction strength
T = 1.0  # Temperature
steps = 1000  # Number of Monte Carlo steps
frames = 100  # Number of frames for animation

# Initialize lattice with random spins on a unit sphere
spins = np.random.normal(size=(L, L, 3))
spins /= np.linalg.norm(spins, axis=-1, keepdims=True)

# Function to calculate energy of a spin configuration
def calculate_energy(spins):
    energy = 0.0
    for i in range(L):
        for j in range(L):
            S = spins[i, j]
            neighbors = (
                spins[(i + 1) % L, j]
                + spins[(i - 1) % L, j]
                + spins[i, (j + 1) % L]
                + spins[i, (j - 1) % L]
            )
            energy -= J * np.dot(S, neighbors)
    return energy / 2.0  # Avoid double counting

# Monte Carlo step using Metropolis algorithm
def monte_carlo_step(spins, T):
    for _ in range(L * L):  # Attempt one update per spin
        i, j = np.random.randint(0, L, size=2)
        S_old = spins[i, j]
        d_theta = np.random.uniform(-np.pi/4, np.pi/4)
        d_phi = np.random.uniform(-np.pi/4, np.pi/4)
        rotation_matrix = np.array([
            [np.cos(d_phi), -np.sin(d_phi), 0],
            [np.sin(d_phi), np.cos(d_phi), 0],
            [0, 0, 1]
        ])
        S_new = rotation_matrix @ S_old
        S_new /= np.linalg.norm(S_new)

        # Calculate energy difference
        neighbors = (
            spins[(i + 1) % L, j]
            + spins[(i - 1) % L, j]
            + spins[i, (j + 1) % L]
            + spins[i, (j - 1) % L]
        )
        dE = J * np.dot(S_new - S_old, neighbors)

        # Metropolis criterion
        if dE < 0 or np.random.rand() < np.exp(-dE / T):
            spins[i, j] = S_new

# Visualization setup
fig, ax = plt.subplots(figsize=(6, 6))
quiver = ax.quiver(
    np.arange(L),
    np.arange(L),
    spins[:, :, 0],
    spins[:, :, 1],
    scale=1,
    scale_units='xy'
)
ax.set_xlim(-1, L)
ax.set_ylim(-1, L)
ax.set_aspect('equal')
ax.axis('off')

# Update function for animation
def update(frame):
    for _ in range(steps // frames):
        monte_carlo_step(spins, T)
    quiver.set_UVC(spins[:, :, 0], spins[:, :, 1])
    return quiver,

# Create animation
anim = FuncAnimation(fig, update, frames=frames, interval=50)
writer = PillowWriter(fps=20)
anim.save("heisenberg_simulation.gif", writer=writer)

plt.close()
